<a href="https://colab.research.google.com/github/cardstdani/sql-storage/blob/main/PostgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
# @title **Click to install dependencies (PostgreSQL DBMS and *faker* library for Python)**
!apt-get update -qq && apt-get install -qq -y postgresql postgresql-contrib
!pip install faker

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Preconfiguring packages ...
Selecting previously unselected package logrotate.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../00-logrotate_3.19.0-1ubuntu1.1_amd64.deb ...
Unpacking logrotate (3.19.0-1ubuntu1.1) ...
Selecting previously unselected package netbase.
Preparing to unpack .../01-netbase_6.3_all.deb ...
Unpacking netbase (6.3) ...
Selecting previously unselected package libcommon-sense-perl:amd64.
Preparing to unpack .../02-libcommon-sense-perl_3.75-2build1_amd64.deb ...
Unpacking libcommon-sense-perl:amd64 (3.75-2build1) ...
Selecting previously unselected package libjson-perl.
Preparing to unpack .../03-libjson-perl_4.04000-1_all.deb ...
Unpacking libjson-perl (4.04000-1) ...
Selecting previously unselected package libtypes-serialiser-perl

In [4]:
# @title **Start PostgreSQL service**
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [ ]:
# @title **Install libraries to support GPU acceleration (optional)**

!apt-get update -qq && apt-get install -qq -y nvidia-cuda-toolkit git make

# 2) Detecta la versión de PostgreSQL y la ruta a pg_config
!PG_VER=$(ls /etc/postgresql)                             # p.ej. "15"
!PG_CONFIG=$(which pg_config)

# 3) Clona y compila PG-Strom
!git clone https://github.com/pg-strom/devel.git pg-strom
!cd pg-strom && make USE_PGXS=1 PG_CONFIG=$PG_CONFIG CUDA_HOME=/usr/local/cuda
!cd pg-strom && make install

# 4) Habilita la librería en postgresql.conf
!echo "shared_preload_libraries = 'pg_strom'" \
  >> /etc/postgresql/$PG_VER/main/postgresql.conf

# 5) Reinicia PostgreSQL para cargar la extensión
!service postgresql restart

**Database creation**

In [5]:
!sudo -u postgres psql -c "CREATE DATABASE ExampleDataBase OWNER postgres;"

CREATE DATABASE


In [ ]:
# @title **Add GPU acceleration support extension to the previous database (optional)**
!sudo -u postgres psql -d exampledatabase -c "CREATE EXTENSION IF NOT EXISTS pg_strom;"

ERROR:  could not open extension control file "/usr/share/postgresql/14/extension/pg_strom.control": No such file or directory


**DDL script execution**

In [6]:
!wget -qO schema.sql https://gist.githubusercontent.com/cardstdani/1247573e1ef2f6ea9ab99b82c5761ad6/raw/schema.sql

!sudo -u postgres psql -q -d exampledatabase -f schema.sql

psql:schema.sql:1: NOTICE:  table "sportsanction" does not exist, skipping
psql:schema.sql:2: NOTICE:  table "poolsanction" does not exist, skipping
psql:schema.sql:3: NOTICE:  table "drivingsanction" does not exist, skipping
psql:schema.sql:4: NOTICE:  table "busticket" does not exist, skipping
psql:schema.sql:5: NOTICE:  table "participation" does not exist, skipping
psql:schema.sql:6: NOTICE:  table "carregistration" does not exist, skipping
psql:schema.sql:7: NOTICE:  table "membership" does not exist, skipping
psql:schema.sql:8: NOTICE:  table "cruisebooking" does not exist, skipping
psql:schema.sql:9: NOTICE:  table "residence" does not exist, skipping
psql:schema.sql:10: NOTICE:  table "entry" does not exist, skipping
psql:schema.sql:11: NOTICE:  table "bustrip" does not exist, skipping
psql:schema.sql:12: NOTICE:  table "team" does not exist, skipping
psql:schema.sql:13: NOTICE:  table "citybus" does not exist, skipping
psql:schema.sql:14: NOTICE:  table "carownership" does not

In [7]:
# @title **Data generation**

import random
from faker import Faker
from datetime import date, datetime, timedelta

# --------------------------------------------------
# CONFIGURACIÓN (ajusta aquí si lo necesitas)
# --------------------------------------------------
SEED = 10 # @param {"type":"integer"}
FACTOR = 100.0 # @param {"type":"number"}
OUTPUT_PATH = '/content/inserts.sql'  # Fichero donde se escribirán los INSERTs


# --------------------------------------------------
# Función de sanitización
# --------------------------------------------------
def safe_str(s, max_len=32):
    """Elimina saltos de línea, escapa comillas simples y trunca."""
    if s is None:
        return None
    clean = s.replace("\n", " ").replace("\r", " ").replace("'", "''").strip()
    return clean[:max_len]


# --------------------------------------------------
# 1) Recuentos base por tabla
# --------------------------------------------------
BASE_COUNTS = {
    'CruiseLine': 5,
    'Person': 50,
    'City': 10,
    'Vehicle': None,  # se calcula más abajo
    'CruiseShip': 5,
    'Bike': 10,
    'Car': 10,
    'DrivingLicenseRequest': 20,
    'DrivingLicense': 15,
    'RejectedDrivingLicense': 5,
    'CarRegistration': 10,
    'CarOwnership': 10,
    'Rental': 10,
    'ShipAssignment': 10,
    'Port': 10,
    'Voyage': 20,
    'CruiseBooking': 20,
    'Residence': 20,
    'Pool': 10,
    'CruisePool': 5,
    'CityPool': 5,
    'OlympicPool': 5,
    'Entry': 20,
    'SwimmingCompetition': 5,
    'Team': 10,
    'Participation': 20,
    'Sanction': 20,
    'DrivingSanction': 10,
    'SportSanction': 5,
    'PoolSanction': 5,
    'Membership': 10,
    'CityBus': 10,
    'BusTrip': 20,
    'BusPass': 10,
    'BusTicket': 20
}

# --------------------------------------------------
# 2) Inicializar PRNG y Faker
# --------------------------------------------------
random.seed(SEED)
faker = Faker()
Faker.seed(SEED)

# --------------------------------------------------
# 3) Calcular recuentos reales
# --------------------------------------------------
counts = {}
for table, base in BASE_COUNTS.items():
    if base is not None:
        counts[table] = int(base * FACTOR)
counts['Vehicle'] = (
        counts['CruiseShip']
        + counts['Bike']
        + counts['Car']
        + counts['CityBus']
)

# --------------------------------------------------
# 4) Variables de estado / llaves foráneas
# --------------------------------------------------
inserts = []
cruise_lines = []
persons = []
cities = []
vehicles = []
cruise_ships = []
bikes = []  # aquí guardaremos los BikeID reales
cars = []
dl_requests = []
dl_licenses = []
ports = []
voyages = []
pools = list(range(1, counts['Pool'] + 1))
cruise_pool_ids = []
city_pool_ids = []
olympic_pool_ids = []
entries = []
competitions = []
teams = []
participations = []
sanctions = []
citybus_plates = []
bustrips = []
buspasses = []

# --------------------------------------------------
# 5) Generación de INSERTs
# --------------------------------------------------

# 5.1 CruiseLine
for _ in range(counts['CruiseLine']):
    name = safe_str(faker.company())
    fd = faker.date_between(date(1950, 1, 1), date.today() - timedelta(days=1))
    phone = random.randint(10 ** 9, 10 ** 10 - 1)
    rating = round(random.random() * 5, 2)
    cruise_lines.append((name, fd))
    inserts.append(
        f"INSERT INTO CruiseLine "
        f"(Name,FoundationDate,ContactPhone,Rating) "
        f"VALUES ('{name}','{fd}',{phone},{rating});"
    )

# 5.2 Person
for pid in range(1, counts['Person'] + 1):
    nm = safe_str(faker.name())
    birth = faker.date_of_birth(minimum_age=18, maximum_age=90)
    email = safe_str(faker.email())
    phone = random.randint(10 ** 9, 10 ** 10 - 1)
    nat = safe_str(faker.country())
    if cruise_lines and random.random() < 0.3:
        cl = random.choice(cruise_lines)
        vals = f"('{nm}','{birth}','{email}',{phone},'{nat}','{cl[0]}','{cl[1]}')"
    else:
        vals = f"('{nm}','{birth}','{email}',{phone},'{nat}',NULL,NULL)"
    inserts.append(
        f"INSERT INTO Person "
        f"(Name,Birth,Email,Phone,Nationality,NameFK,FoundationDateFK) "
        f"VALUES {vals};"
    )
    persons.append(pid)

# 5.3 City
for cid in range(1, counts['City'] + 1):
    cname = safe_str(faker.city())
    country = safe_str(faker.country())
    pop = random.randint(0, 10_000_000)
    area = round(random.uniform(10, 5000), 2)
    lat = round(random.uniform(-90, 90), 6)
    lon = round(random.uniform(-180, 180), 6)
    inserts.append(
        f"INSERT INTO City "
        f"(Name,Country,Population,Area,Latitude,Longitude) "
        f"VALUES ('{cname}','{country}',{pop},{area},{lat},{lon});"
    )
    cities.append(cid)

# 5.4 Vehicle
colors = ['red', 'green', 'blue', 'yellow', 'black', 'white']
for vid in range(1, counts['Vehicle'] + 1):
    mdl = safe_str(faker.word())
    wgt = round(random.uniform(500, 3000), 2)
    col = random.choice(colors)
    odo = round(random.uniform(0, 200000), 2)
    inserts.append(
        f"INSERT INTO Vehicle "
        f"(Model,Weight,Color,Odometer) "
        f"VALUES ('{mdl}',{wgt},'{col}',{odo});"
    )
    vehicles.append(vid)

# repartir VehicleIDs
idx = 0
cs_ids = vehicles[idx: idx + counts['CruiseShip']];
idx += counts['CruiseShip']
bike_vids = vehicles[idx: idx + counts['Bike']];
idx += counts['Bike']
car_vids = vehicles[idx: idx + counts['Car']];
idx += counts['Car']
bus_vids = vehicles[idx: idx + counts['CityBus']]

# 5.5 CruiseShip
classes = ['first', 'second', 'third', 'economy']
for vid in cs_ids:
    spd = round(random.uniform(10, 30), 2)
    lng = round(random.uniform(100, 400), 2)
    cap = random.randint(100, 5000)
    cls_t = random.choice(classes)
    inserts.append(
        f"INSERT INTO CruiseShip "
        f"(Speed,Length,PassengerCapacity,Class,VehicleID) "
        f"VALUES ({spd},{lng},{cap},'{cls_t}',{vid});"
    )
    cruise_ships.append(vid)

# 5.6 Bike
bike_pk_counter = 1
for vid in bike_vids:
    ele = random.choice([True, False])
    fol = random.choice([True, False])
    lig = random.choice([True, False])
    fh = round(random.uniform(14, 24), 2)
    inserts.append(
        f"INSERT INTO Bike "
        f"(Electric,Foldable,HasLights,FrameHeight,VehicleID) "
        f"VALUES ({ele},{fol},{lig},{fh},{vid});"
    )
    bikes.append(bike_pk_counter)
    bike_pk_counter += 1

# 5.7 Car
fuels = ['gas', 'diesel', 'electric', 'hybrid', 'hydrogen']
for vid in car_vids:
    plate = safe_str(faker.bothify('???-#####'))
    fuel = random.choice(fuels)
    doorc = random.randint(2, 5)
    trunk = random.randint(100, 1000)
    hp = random.randint(60, 500)
    doors = random.randint(1, 5)
    ac = random.choice([True, False])
    inserts.append(
        f"INSERT INTO Car "
        f"(Plate,FuelType,DoorCount,TrunkCapacity,HorsePower,Doors,AirConditioning,VehicleID) "
        f"VALUES ('{plate}','{fuel}',{doorc},{trunk},{hp},{doors},{ac},{vid});"
    )
    cars.append(plate)

# 5.8 DrivingLicenseRequest
for lid in range(1, counts['DrivingLicenseRequest'] + 1):
    rd = faker.date_between(date.today() - timedelta(days=365), date.today())
    fee = round(random.uniform(20, 200), 2)
    p = random.choice(persons)
    inserts.append(
        f"INSERT INTO DrivingLicenseRequest "
        f"(RequestDate,Fee,PersonFK) "
        f"VALUES ('{rd}',{fee},{p});"
    )
    dl_requests.append(lid)

# 5.9 DrivingLicense
for lid in dl_requests[:counts['DrivingLicense']]:
    ad = faker.date_between(date.today() - timedelta(days=365), date.today())
    pts = random.randint(0, 15)
    inserts.append(
        f"INSERT INTO DrivingLicense "
        f"(LicenseID,ApprovalDate,Points) "
        f"VALUES ({lid},'{ad}',{pts});"
    )
    dl_licenses.append(lid)

# 5.10 RejectedDrivingLicense
for lid in dl_requests[
           counts['DrivingLicense']:
           counts['DrivingLicense'] + counts['RejectedDrivingLicense']
           ]:
    rej = faker.date_between(date.today() - timedelta(days=365), date.today())
    reap = faker.date_between(rej, rej + timedelta(days=365))
    reason = safe_str(faker.word())
    inserts.append(
        f"INSERT INTO RejectedDrivingLicense "
        f"(LicenseID,RejectionDate,ReapplicationDate,Reason) "
        f"VALUES ({lid},'{rej}','{reap}','{reason}');"
    )

# 5.11 CarRegistration
for _ in range(counts['CarRegistration']):
    reg = faker.date_between(date.today() - timedelta(days=365), date.today())
    exp = faker.date_between(reg + timedelta(days=1), reg + timedelta(days=365))
    pl = random.choice(cars)
    li = random.choice(dl_licenses)
    inserts.append(
        f"INSERT INTO CarRegistration "
        f"(RegistrationDate,ExpirationDate,PlateFK,LicenseFK) "
        f"VALUES ('{reg}','{exp}','{pl}',{li});"
    )

# 5.12 CarOwnership
own_types = ['buy', 'rental', 'lease']
for _ in range(counts['CarOwnership']):
    bd = faker.date_time_between(start_date='-2y', end_date='now')
    price = round(random.uniform(10000, 100000), 2)
    we = (bd + timedelta(days=random.randint(365, 3650))).date()
    typ = random.choice(own_types)
    pl = random.choice(cars)
    p = random.choice(persons)
    inserts.append(
        f"INSERT INTO CarOwnership "
        f"(BuyDate,BuyPrice,WarrantyEndDate,OwnershipType,PlateFK,PersonFK) "
        f"VALUES ('{bd}',{price},'{we}','{typ}','{pl}',{p});"
    )

# 5.13 Rental (usa BikeID válidos)
for _ in range(counts['Rental']):
    st = faker.date_time_between(start_date='-1y', end_date='now')
    dur = random.randint(0, 48)
    hrp = round(random.uniform(5, 50), 2)
    bk = random.choice(bikes)  # ahora BikeID real
    p = random.choice(persons)
    inserts.append(
        f"INSERT INTO Rental "
        f"(StartTimestamp,Duration,HourPrice,BikeFK,PersonFK) "
        f"VALUES ('{st}',{dur},{hrp},{bk},{p});"
    )

# 5.14 ShipAssignment
for _ in range(counts['ShipAssignment']):
    sd = faker.date_between(date.today() - timedelta(days=365 * 5), date.today())
    ed = faker.date_between(sd, sd + timedelta(days=365))
    cl_line = random.choice(cruise_lines)
    ship = random.choice(cruise_ships)
    inserts.append(
        f"INSERT INTO ShipAssignment "
        f"(StartDate,EndDate,NameFK,FoundationDateFK,ShipFK) "
        f"VALUES ('{sd}','{ed}','{cl_line[0]}','{cl_line[1]}',{ship});"
    )

# 5.15 Port
for _ in range(counts['Port']):
    pname = safe_str(faker.city())
    tc = random.randint(0, 10)
    ms = random.randint(100, 500)
    area = round(random.uniform(1, 100), 2)
    cfk = random.choice(cities)
    inserts.append(
        f"INSERT INTO Port "
        f"(Name,TerminalCount,MaxShipLength,Area,CityFK) "
        f"VALUES ('{pname}',{tc},{ms},{area},{cfk});"
    )
    ports.append((pname, cfk))

# 5.16 Voyage
for _ in range(counts['Voyage']):
    dep_name, dep_city = random.choice(ports)
    arr_name, arr_city = random.choice([p for p in ports if p != (dep_name, dep_city)])
    dd = faker.date_between(date.today() - timedelta(days=365), date.today())
    ad = faker.date_between(dd, dd + timedelta(days=30))
    dist = round(random.uniform(10, 10000), 2)
    sp = random.choice(cruise_ships)
    inserts.append(
        f"INSERT INTO Voyage "
        f"(DepartureDate,ArrivalDate,Distance,"
        f"DepartureNameFK,DepartureCityFK,"
        f"ArrivalNameFK,ArrivalCityFK,ShipFK) "
        f"VALUES ('{dd}','{ad}',{dist},"
        f"'{dep_name}',{dep_city},"
        f"'{arr_name}',{arr_city},{sp});"
    )
    voyages.append((dd, ad, dep_name, dep_city, arr_name, arr_city, sp))

# 5.17 CruiseBooking
pay_methods = ['card', 'paypal', 'bank', 'cash', 'mobile']
for _ in range(counts['CruiseBooking']):
    bk_date = faker.date_between(date.today() - timedelta(days=365), date.today())
    cab = random.randint(1, 5000)
    price = round(random.uniform(100, 10000), 2)
    pm = random.choice(pay_methods)
    per = random.choice(persons)
    voy_key = random.choice(voyages)
    inserts.append(
        f"INSERT INTO CruiseBooking "
        f"(BookingDate,CabinNumber,Price,PaymentMethod,PersonFK,"
        f"DepartureDateFK,ArrivalDateFK,DepartureNameFK,DepartureCityFK,"
        f"ArrivalNameFK,ArrivalCityFK,ShipFK) VALUES "
        f"('{bk_date}',{cab},{price},'{pm}',{per},"
        f"'{voy_key[0]}','{voy_key[1]}','{voy_key[2]}',{voy_key[3]},"
        f"'{voy_key[4]}',{voy_key[5]},{voy_key[6]});"
    )

# 5.18 Residence
for _ in range(counts['Residence']):
    sd = faker.date_between(date.today() - timedelta(days=365 * 5), date.today())
    if random.random() < 0.8:
        ed = faker.date_between(sd, sd + timedelta(days=365 * 2))
        ed_s = f"'{ed}'"
    else:
        ed_s = "NULL"
    addr = safe_str(faker.street_address())
    psn = random.choice(persons)
    cfk = random.choice(cities)
    inserts.append(
        f"INSERT INTO Residence "
        f"(StartDate,EndDate,Address,PersonFK,CityFK) "
        f"VALUES ('{sd}',{ed_s},'{addr}',{psn},{cfk});"
    )

# 5.19 Pool
statuses = ['open', 'closed', 'maintenance', 'renovation']
for pid in pools:
    nm = safe_str(faker.company())
    addr = safe_str(faker.address())
    mind = random.randint(0, 5)
    maxd = mind + random.randint(0, 10)
    st = random.choice(statuses)
    c_fk = random.choice(cities)
    inserts.append(
        f"INSERT INTO Pool "
        f"(PoolID,Name,Address,MinDepth,MaxDepth,Status,CityFK) "
        f"VALUES ({pid},'{nm}','{addr}',{mind},{maxd},'{st}',{c_fk});"
    )

# 5.20 CruisePool
cruise_pool_ids = random.sample(pools, counts['CruisePool'])
for pid in cruise_pool_ids:
    dn = random.randint(0, 10)
    mc = random.randint(0, 500)
    wt = round(random.uniform(20, 35), 2)
    sc = random.randint(0, 5)
    sh = random.choice(cruise_ships)
    inserts.append(
        f"INSERT INTO CruisePool "
        f"(PoolID,DeckNumber,MaxCapacity,WaterTemperature,SlideCount,ShipFK) "
        f"VALUES ({pid},{dn},{mc},{wt},{sc},{sh});"
    )

# 5.21 CityPool
city_pool_ids = random.sample(pools, counts['CityPool'])
for pid in city_pool_ids:
    mc = random.randint(0, 5000)
    ab = round(random.uniform(1000, 1e6), 2)
    af = safe_str(faker.word())
    fw = random.choice([True, False])
    inserts.append(
        f"INSERT INTO CityPool "
        f"(PoolID,MaxCapacity,AnnualBudget,AccessibilityFeatures,FreeWifi) "
        f"VALUES ({pid},{mc},{ab},'{af}',{fw});"
    )

# 5.22 OlympicPool
olympic_pool_ids = random.sample(pools, counts['OlympicPool'])
for pid in olympic_pool_ids:
    spcap = random.randint(0, 10000)
    lanes = random.randint(1, 20)
    inserts.append(
        f"INSERT INTO OlympicPool "
        f"(PoolID,SpectatorMaxCapacity,CompetitionLanes) "
        f"VALUES ({pid},{spcap},{lanes});"
    )

# 5.23 Entry
for _ in range(counts['Entry']):
    ts = faker.date_time_between(start_date='-1y', end_date='now')
    price = round(random.uniform(5, 100), 2)
    pm = random.choice(pay_methods)
    disc = round(random.uniform(0, price), 2)
    dur = random.randint(1, 300)
    per = random.choice(persons)
    pl = random.choice(city_pool_ids)
    inserts.append(
        f"INSERT INTO Entry "
        f"(EntryTimestamp,Price,PaymentMethod,AppliedDiscount,Duration,PersonFK,PoolFK) "
        f"VALUES ('{ts}',{price},'{pm}',{disc},{dur},{per},{pl});"
    )
    entries.append((ts, per, pl))

# 5.24 SwimmingCompetition
for _ in range(counts['SwimmingCompetition']):
    nm = safe_str(faker.word())
    sd = faker.date_between(date.today() - timedelta(days=365), date.today())
    ed = faker.date_between(sd, sd + timedelta(days=30))
    rt = round(random.uniform(20, 200), 2)
    prize = random.randint(0, 100_000)
    inserts.append(
        f"INSERT INTO SwimmingCompetition "
        f"(Name,StartDate,EndDate,RecordTime,PrizeAmount) "
        f"VALUES ('{nm}','{sd}','{ed}',{rt},{prize});"
    )
    competitions.append((nm, sd, ed))

# 5.25 Team
sports = ['waterpolo', 'swimming', 'diving']
for _ in range(counts['Team']):
    nm = safe_str(faker.word())
    cd = faker.date_between(date(1950, 1, 1), date.today() - timedelta(days=1))
    cloth = random.choice(colors)
    sport = random.choice(sports)
    budg = random.randint(0, 1_000_000)
    em = safe_str(faker.email())
    coach = random.choice(persons)
    home = random.choice(olympic_pool_ids)
    inserts.append(
        f"INSERT INTO Team "
        f"(Name,CreationDate,ClothColor,Sport,Budget,ContactEmail,CoachFK,HomePoolFK) "
        f"VALUES ('{nm}','{cd}','{cloth}','{sport}',{budg},'{em}',{coach},{home});"
    )
    teams.append((nm, coach))

# 5.26 Participation
all_pairs = [(c, t) for c in competitions for t in teams]
for comp, team in random.sample(all_pairs, counts['Participation']):
    rd = faker.date_between(comp[1], comp[2])
    rk = random.randint(1, 100)
    rt = round(random.uniform(20, 200), 2)
    inserts.append(
        f"INSERT INTO Participation "
        f"(RegistrationDate,Rank,RecordedTime,"
        f"NameFK,StartDateFK,EndDateFK,TeamNameFK,CoachFK) VALUES "
        f"('{rd}',{rk},{rt},'{comp[0]}','{comp[1]}','{comp[2]}','{team[0]}',{team[1]});"
    )
    participations.append((comp[0], comp[1], comp[2], team[0], team[1]))

# 5.27 Sanction
for sid in range(1, counts['Sanction'] + 1):
    amt = round(random.uniform(0, 1000), 2)
    desc = safe_str(faker.word())
    idt = faker.date_between(date.today() - timedelta(days=365), date.today())
    if random.random() < 0.5:
        edt = faker.date_between(idt, idt + timedelta(days=365))
        edt_s = f"'{edt}'"
    else:
        edt_s = "NULL"
    st = random.choice(['created', 'active', 'expired'])
    inserts.append(
        f"INSERT INTO Sanction "
        f"(Amount,Description,IssueDate,ExpirationDate,Status) "
        f"VALUES ({amt},'{desc}','{idt}',{edt_s},'{st}');"
    )
    sanctions.append(sid)

# 5.28 DrivingSanction
dr_ids = random.sample(sanctions, counts['DrivingSanction'])
for sid in dr_ids:
    rs = round(random.uniform(0, 200), 2)
    pd = random.randint(0, 15)
    sdur = random.randint(0, 365)
    perm = random.choice([True, False])
    lic = random.choice(dl_licenses)
    inserts.append(
        f"INSERT INTO DrivingSanction "
        f"(SanctionID,RecordedSpeed,PointsDeducted,SuspensionDays,PermanentSuspension,LicenseFK) "
        f"VALUES ({sid},{rs},{pd},{sdur},{perm},{lic});"
    )

# 5.29 SportSanction
remaining = [s for s in sanctions if s not in dr_ids]
sp_ids = random.sample(remaining, counts['SportSanction'])
for sid in sp_ids:
    part = random.choice(participations)
    susp_c = random.randint(0, 10)
    ref = safe_str(faker.name())
    inserts.append(
        f"INSERT INTO SportSanction "
        f"(SanctionID,SuspendedCompetitions,RefereeName,"
        f"NameFK,StartDateFK,EndDateFK,TeamNameFK,CoachFK) VALUES "
        f"({sid},{susp_c},'{ref}','{part[0]}','{part[1]}','{part[2]}','{part[3]}',{part[4]});"
    )

# 5.30 PoolSanction
remaining = [s for s in remaining if s not in sp_ids]
ps_ids = random.sample(remaining, counts['PoolSanction'])
for sid in ps_ids:
    if random.random() < 0.5:
        bs = faker.date_between(date.today() - timedelta(days=365), date.today())
        be = faker.date_between(bs, bs + timedelta(days=365))
        bs_s, be_s = f"'{bs}'", f"'{be}'"
    else:
        bs_s, be_s = "NULL", "NULL"
    comp = random.randint(0, 1000)
    cs = random.randint(0, 500)
    ent = random.choice(entries)
    inserts.append(
        f"INSERT INTO PoolSanction "
        f"(SanctionID,BanStartDate,BanEndDate,CompensationRequired,"
        f"CommunityServiceHours,EntryFK,PersonFK,PoolFK) VALUES "
        f"({sid},{bs_s},{be_s},{comp},{cs},'{ent[0]}',{ent[1]},{ent[2]});"
    )

# 5.31 Membership
for _ in range(counts['Membership']):
    jd = faker.date_between(date.today() - timedelta(days=365 * 3), date.today())
    if random.random() < 0.8:
        ld = faker.date_between(jd, jd + timedelta(days=365 * 2))
        ld_s = f"'{ld}'"
    else:
        ld_s = "NULL"
    fee = random.randint(0, 1000)
    pf = random.choice(['monthly', 'anual', 'weekly', 'quarterly'])
    ar = random.choice([True, False])
    psn = random.choice(persons)
    tm = random.choice(teams)
    inserts.append(
        f"INSERT INTO Membership "
        f"(JoinDate,LeaveDate,FeeAmount,PaymentFrequency,AutoRenewal,PersonFK,TeamNameFK,CoachFK) "
        f"VALUES ('{jd}',{ld_s},{fee},'{pf}',{ar},{psn},'{tm[0]}',{tm[1]});"
    )

# 5.32 CityBus
for vid in bus_vids:
    plate = safe_str(faker.bothify('BUS-#####'))
    rn = random.randint(0, 100)
    seats = random.randint(10, 100)
    fw = random.choice([True, False])
    dr = random.choice(persons)
    cfk = random.choice(cities)
    inserts.append(
        f"INSERT INTO CityBus "
        f"(Plate,RouteNumber,Seats,FreeWifi,VehicleID,DriverFK,CityFK) "
        f"VALUES ('{plate}',{rn},{seats},{fw},{vid},{dr},{cfk});"
    )
    citybus_plates.append(plate)

# 5.33 BusTrip
for _ in range(counts['BusTrip']):
    dt = faker.date_between(date.today() - timedelta(days=365), date.today())
    sa = safe_str(faker.street_address())
    ea = safe_str(faker.street_address())
    duration = random.randint(5, 120)
    pl = random.choice(citybus_plates)
    inserts.append(
        f"INSERT INTO BusTrip "
        f"(TripDate,StartAddress,EndAddress,Duration,PlateFK) "
        f"VALUES ('{dt}','{sa}','{ea}',{duration},'{pl}');"
    )
    bustrips.append((dt, sa, ea, pl))

# 5.34 BusPass
modalities = ['single', 'round_trip', 'daily', 'weekly', 'monthly', 'annual']
for pid in range(1, counts['BusPass'] + 1):
    idt = faker.date_between(date.today() - timedelta(days=365), date.today())
    edt = faker.date_between(idt + timedelta(days=1), idt + timedelta(days=365))
    mod = random.choice(modalities)
    rt = random.randint(0, 100)
    psn = random.choice(persons)
    inserts.append(
        f"INSERT INTO BusPass "
        f"(IssueDate,ExpirationDate,Modality,RemainingTrips,PersonFK) "
        f"VALUES ('{idt}','{edt}','{mod}',{rt},{psn});"
    )
    buspasses.append(pid)

# 5.35 BusTicket
for _ in range(counts['BusTicket']):
    it = faker.date_time_between(start_date='-1y', end_date='now')
    trip = random.choice(bustrips)
    if random.random() < 0.5:
        psn = random.choice(persons)
        pfk = None
    else:
        psn = None
        pfk = random.choice(buspasses)
    psn_str = str(psn) if psn is not None else "NULL"
    pfk_str = str(pfk) if pfk is not None else "NULL"
    inserts.append(
        f"INSERT INTO BusTicket "
        f"(IssueTime,TripDateFK,StartAddressFK,EndAddressFK,PlateFK,PersonFK,PassFK) "
        f"VALUES ('{it}','{trip[0]}','{trip[1]}','{trip[2]}','{trip[3]}',{psn_str},{pfk_str});"
    )

# --------------------------------------------------
# 6) Escribir fichero de salida
# --------------------------------------------------
with open(OUTPUT_PATH, 'w', encoding='utf-8') as f:
    for stmt in inserts:
        f.write(stmt + "\n")

print(f"¡Listo! {len(inserts)} INSERTs generados en {OUTPUT_PATH}")


¡Listo! 46500 INSERTs generados en /content/inserts.sql


In [8]:
!sudo -u postgres psql -q -d exampledatabase -f inserts.sql

psql:inserts.sql:40584: ERROR:  duplicate key value violates unique constraint "citybus_pkey"
DETAIL:  Key (plate)=(BUS-17083) already exists.
psql:inserts.sql:40646: ERROR:  duplicate key value violates unique constraint "citybus_pkey"
DETAIL:  Key (plate)=(BUS-37406) already exists.
psql:inserts.sql:40831: ERROR:  duplicate key value violates unique constraint "citybus_pkey"
DETAIL:  Key (plate)=(BUS-82022) already exists.
psql:inserts.sql:40868: ERROR:  duplicate key value violates unique constraint "citybus_pkey"
DETAIL:  Key (plate)=(BUS-28464) already exists.
psql:inserts.sql:41269: ERROR:  duplicate key value violates unique constraint "citybus_pkey"
DETAIL:  Key (plate)=(BUS-50110) already exists.
psql:inserts.sql:41408: ERROR:  duplicate key value violates unique constraint "citybus_pkey"
DETAIL:  Key (plate)=(BUS-96796) already exists.
psql:inserts.sql:41461: ERROR:  duplicate key value violates unique constraint "citybus_pkey"
DETAIL:  Key (plate)=(BUS-55875) already exists.

In [9]:
!sudo -u postgres psql -P pager=off -d exampledatabase -c "SELECT 'cruiseline' AS table_name, COUNT(*) AS row_count FROM cruiseline UNION ALL SELECT 'person', COUNT(*) FROM person UNION ALL SELECT 'vehicle', COUNT(*) FROM vehicle UNION ALL SELECT 'cruiseship', COUNT(*) FROM cruiseship UNION ALL SELECT 'bike', COUNT(*) FROM bike UNION ALL SELECT 'car', COUNT(*) FROM car UNION ALL SELECT 'drivinglicenserequest', COUNT(*) FROM drivinglicenserequest UNION ALL SELECT 'drivinglicense', COUNT(*) FROM drivinglicense UNION ALL SELECT 'rejecteddrivinglicense', COUNT(*) FROM rejecteddrivinglicense UNION ALL SELECT 'carregistration', COUNT(*) FROM carregistration UNION ALL SELECT 'carownership', COUNT(*) FROM carownership UNION ALL SELECT 'rental', COUNT(*) FROM rental UNION ALL SELECT 'shipassignment', COUNT(*) FROM shipassignment UNION ALL SELECT 'city', COUNT(*) FROM city UNION ALL SELECT 'port', COUNT(*) FROM port UNION ALL SELECT 'voyage', COUNT(*) FROM voyage UNION ALL SELECT 'cruisebooking', COUNT(*) FROM cruisebooking UNION ALL SELECT 'residence', COUNT(*) FROM residence UNION ALL SELECT 'pool', COUNT(*) FROM pool UNION ALL SELECT 'cruisepool', COUNT(*) FROM cruisepool UNION ALL SELECT 'citypool', COUNT(*) FROM citypool UNION ALL SELECT 'olympicpool', COUNT(*) FROM olympicpool UNION ALL SELECT 'entry', COUNT(*) FROM entry UNION ALL SELECT 'swimmingcompetition', COUNT(*) FROM swimmingcompetition UNION ALL SELECT 'team', COUNT(*) FROM team UNION ALL SELECT 'participation', COUNT(*) FROM participation UNION ALL SELECT 'sanction', COUNT(*) FROM sanction UNION ALL SELECT 'drivingsanction', COUNT(*) FROM drivingsanction UNION ALL SELECT 'sportsanction', COUNT(*) FROM sportsanction UNION ALL SELECT 'poolsanction', COUNT(*) FROM poolsanction UNION ALL SELECT 'membership', COUNT(*) FROM membership UNION ALL SELECT 'citybus', COUNT(*) FROM citybus UNION ALL SELECT 'bustrip', COUNT(*) FROM bustrip UNION ALL SELECT 'buspass', COUNT(*) FROM buspass UNION ALL SELECT 'busticket', COUNT(*) FROM busticket;"

       table_name       | row_count 
------------------------+-----------
 cruiseline             |       500
 person                 |      5000
 vehicle                |      3500
 cruiseship             |       500
 bike                   |      1000
 car                    |      1000
 drivinglicenserequest  |      2000
 drivinglicense         |      1500
 rejecteddrivinglicense |       500
 carregistration        |      1000
 carownership           |      1000
 rental                 |      1000
 shipassignment         |      1000
 city                   |      1000
 port                   |      1000
 voyage                 |      2000
 cruisebooking          |      2000
 residence              |      2000
 pool                   |      1000
 cruisepool             |       500
 citypool               |       500
 olympicpool            |       500
 entry                  |      2000
 swimmingcompetition    |       500
 team                   |      1000
 participation          | 

In [10]:
# @title **Query title**
sql = """
WITH VoyageDistance AS (
  SELECT *
  FROM Voyage
  WHERE Distance > 1000
)
SELECT DepartureDate, ArrivalDate, Distance
FROM VoyageDistance
WHERE DepartureDate BETWEEN '2025-01-01' AND '2025-06-30';
"""

!sudo -u postgres psql -P pager=off -d exampledatabase -c "$sql"

 departuredate | arrivaldate | distance 
---------------+-------------+----------
 2025-01-05    | 2025-01-29  |  1133.61
 2025-06-01    | 2025-06-26  |  4206.34
 2025-02-10    | 2025-02-20  |   3013.2
 2025-04-01    | 2025-04-26  |  8770.26
 2025-05-14    | 2025-06-06  |  8553.99
 2025-05-02    | 2025-05-15  |  9565.82
 2025-05-14    | 2025-06-07  |  2246.27
 2025-06-15    | 2025-07-10  |  6834.22
 2025-06-30    | 2025-07-12  |  1809.17
 2025-02-23    | 2025-03-20  |  5543.18
 2025-01-10    | 2025-01-20  |  2345.52
 2025-02-24    | 2025-02-26  |  3050.08
 2025-04-14    | 2025-04-17  |  5864.66
 2025-02-09    | 2025-02-22  |   8332.5
 2025-03-04    | 2025-03-26  |  8904.89
 2025-01-04    | 2025-01-14  |  3373.91
 2025-06-07    | 2025-07-01  |  6175.91
 2025-01-05    | 2025-02-01  |  5146.98
 2025-04-06    | 2025-04-22  |  7124.28
 2025-02-03    | 2025-02-19  |  6018.62
 2025-06-08    | 2025-07-07  |  9310.71
 2025-03-15    | 2025-03-22  |  4338.64
 2025-02-27    | 2025-03-13  |  1281.3